## Real time chimp call detection

In [1]:
import numpy as np
import time
from pydub import AudioSegment
import random
import sys
import io
import os
import glob
import pyaudio
import librosa
import IPython
from td_utils import *
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
# To generate wav file from np array.
from scipy.io.wavfile import write
%matplotlib inline

#### Model

In [3]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

d:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:

inf_model = Sequential()

inf_model.add(Dense(256, input_shape=(40,)))
inf_model.add(Activation('relu'))
inf_model.add(Dropout(0.5))

inf_model.add(Dense(256))
inf_model.add(Activation('relu'))
inf_model.add(Dropout(0.5))

# try_model.add(Dense(256))
# try_model.add(Activation('relu'))
# try_model.add(Dropout(0.5))

inf_model.add(Dense(256))
inf_model.add(Activation('relu'))
inf_model.add(Dropout(0.5))

inf_model.add(Dense(12))
inf_model.add(Activation('softmax'))

inf_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [7]:
inf_model = load_model("my_model.h5")
inf_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
__________

In [8]:
def load_sound_files(file_paths):

    X,sr = librosa.load(file_paths, sr=11025, res_type='kaiser_fast')
    mfccs=np.mean(librosa.feature.mfcc(y=X,sr=sr,n_mfcc=40).T,axis=0)

    return mfccs

In [40]:
import pyaudio
import wave
import os

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 11025
RECORD_SECONDS = 20
WAVE_OUTPUT_FILENAME = "output"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")


for j in range(0,RECORD_SECONDS):
    frames = []
    for i in range(int(RATE / CHUNK * j), int(RATE / CHUNK * (j+4))):
        data = stream.read(CHUNK)
        frames.append(data)
    wf = wave.open("output_{}.wav".format(j), 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    inf = []
    inf.append(load_sound_files("output_{}.wav".format(j)))
    inf_pred = inf_model.predict_classes(np.array(inf))
    print(inf_pred)
    if inf_pred[0] == 10:
        print("Chimp!")
        
print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
[6]
[6]
[6]
[2]
[10]
Chimp!
[6]
[6]
[6]
[3]
[10]
Chimp!
[3]
[3]
[6]
[6]
[6]
[6]
[6]
[6]
[6]
[6]
* done recording


#### Recording from mic

#### Get audio stream